# Inport needed package

- import os, sys # to add the parent directory to the path

In [1]:
import os
import sys
import time

- Using torchvision to create a dataset

In [2]:
# Using torchvision to create a dataset
import cv2
from torchvision import transforms
import torch
from torch.utils.data import random_split, DataLoader
import torchvision

import pandas as pd

- import self library

In [3]:
from train.trainer import ClassifierTrainer as Trainer
import dataset as ds  # type: ignore
import model as md  # type: ignore

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define classification train process

1. Define place where the model is saved

In [4]:
time_str = time.strftime("%Y%m%d_%H%M%S")

2. Define train function

In [5]:
def train_process(dataset, model):
  # define batch_size
  batch_size = 64

  # init train val test ds
  train_val_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_val_size
  train_ds, test_ds = random_split(dataset, [train_val_size, test_size])

  # define optimizer using Adam and loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  trainer = Trainer(model, optimizer, loss_fn, random_seed_value=86)
  print('device: ', trainer.device)
  avg_loss, metric = trainer.cross_validate(train_ds, k=5, epochs=10, batch_size=batch_size)
  print('avg_loss: ', avg_loss)

  # score model
  test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
  model_scored = trainer.score(test_dataloader)
  print(f'model_scored: {model_scored:.4f}, avg_accuracy: {100*(1 - model_scored):.4f}')

  # return model scored, train_avg_lost
  return model_scored, avg_loss

3. execute progress

- define the model

In [6]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

- Define tested datasets

In [7]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Train all defined model on each registered datasets

In [8]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for name, dataset in datasets.items():
	for model in models:
		print(f'Running {name} dataset with {model.__class__.__name__}')
		# do the train
		start_time = time.time()
		scored, loss = train_process(dataset, model)
		end_time = time.time()
		total_time = end_time - start_time
		print(f'Finished {name} dataset with {model.__class__.__name__}')
		print('----------------------')

		# save the result
		result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model.__class__.__name__],
			'dataset': [name],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
		})], ignore_index=True)

print('Finished all datasets')

# print the result
print(result_df)


Running gi4e_full dataset with ResNet
device:  cuda
Fold 1/5:
Epoch 1/10, Train Loss: 2.3302, Test Loss: 0.3474, Total Time: 00 hours, 00 minutes, 25 seconds
Epoch 2/10, Train Loss: 0.0113, Test Loss: 0.0073, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 3/10, Train Loss: 0.0010, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 4/10, Train Loss: 0.0005, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 5/10, Train Loss: 0.0002, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 6/10, Train Loss: 0.0002, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 

C:\Users\nnvuf\AppData\Local\Temp\ipykernel_6592\454607160.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


Epoch 1/10, Train Loss: 1.8810, Test Loss: 1.7562, Total Time: 00 hours, 00 minutes, 27 seconds
Epoch 2/10, Train Loss: 0.0233, Test Loss: 0.0677, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.0039, Test Loss: 0.0029, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 4/10, Train Loss: 0.0015, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 5/10, Train Loss: 0.0008, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 6/10, Train Loss: 0.0006, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 7/10, Train Loss: 0.0005, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 8/10, Train Loss: 0.0003, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 9/10, Train Loss: 0.0003, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 10/10, Train Loss: 0.0003, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 20 seconds
Fold 1/5, Total Test Loss: 0.0003, Fold

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Fold 1/5, Total Test Loss: 0.0753, Fold accuracy: 92.4704
Fold 2/5:
Epoch 1/10, Train Loss: 0.2183, Test Loss: 0.0566, Total Time: 00 hours, 00 minutes, 37 seconds
Epoch 2/10, Train Loss: 0.1481, Test Loss: 0.0172, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.0542, Test Loss: 0.0433, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 4/10, Train Loss: 0.0324, Test Loss: 0.0032, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 5/10, Train Loss: 0.0382, Test Loss: 0.0032, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 6/10, Train Loss: 0.0153, Test Loss: 0.0024, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 7/10, Train Loss: 0.0232, Test Loss: 0.0361, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 8/10, Train Loss: 0.1000, Test Loss: 0.4209, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 9/10, Train Loss: 0.5311, Test Loss: 0.2061, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.3432, Test Loss: 0.0545, Total Time: 00 h

...

Fold 2/5, Total Test Loss: 0.1298, Fold accuracy: 94.5463
Fold 3/5:
Epoch 1/10, Train Loss: 0.1144, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 2/10, Train Loss: 0.0457, Test Loss: 0.0076, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.0318, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 4/10, Train Loss: 0.0253, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 5/10, Train Loss: 0.0145, Test Loss: 0.0011, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 6/10, Train Loss: 0.0171, Test Loss: 0.0873, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 7/10, Train Loss: 0.1577, Test Loss: 0.3540, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 8/10, Train Loss: 0.1349, Test Loss: 0.0141, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 9/10, Train Loss: 0.0463, Test Loss: 0.0177, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.0406, Test Loss: 0.0594, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 3/5, Total Test Loss: 0.1893, Fold accuracy: 94.0571
Fold 4/5:
Epoch 1/10, Train Loss: 0.1170, Test Loss: 0.0227, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 2/10, Train Loss: 0.0619, Test Loss: 0.0077, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.0221, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 4/10, Train Loss: 0.0252, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 5/10, Train Loss: 0.0441, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 22 seconds
Epoch 6/10, Train Loss: 0.0876, Test Loss: 0.0051, Total Time: 00 hours, 00 minutes, 23 seconds
Epoch 7/10, Train Loss: 0.0952, Test Loss: 0.2752, Total Time: 00 hours, 00 minutes, 23 seconds
Epoch 8/10, Train Loss: 0.0858, Test Loss: 0.0017, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 9/10, Train Loss: 0.1133, Test Loss: 0.0394, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.0560, Test Loss: 0.0596, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 4/5, Total Test Loss: 0.2488, Fold accuracy: 94.0423
Fold 5/5:
Epoch 1/10, Train Loss: 0.1062, Test Loss: 0.0368, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 2/10, Train Loss: 0.1721, Test Loss: 0.0048, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.6862, Test Loss: 0.0886, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 4/10, Train Loss: 0.1793, Test Loss: 0.0702, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 5/10, Train Loss: 0.1211, Test Loss: 0.0027, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 6/10, Train Loss: 0.0224, Test Loss: 0.0009, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 7/10, Train Loss: 0.0491, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 8/10, Train Loss: 0.0607, Test Loss: 0.0135, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 9/10, Train Loss: 0.0369, Test Loss: 0.0024, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.0313, Test Loss: 0.0003, Total Time: 00 h

4. print the result

In [9]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000008     99.999246   
1           gi4e_full  DenseNet  0.000084     99.997677   
2           gi4e_full       VGG  0.049829     97.527197   
3       gi4e_raw_eyes    ResNet  0.011788     94.142118   
4       gi4e_raw_eyes  DenseNet  0.001661     93.929313   
5       gi4e_raw_eyes       VGG  0.075721     74.313858   
6  gi4e_detected_eyes    ResNet  0.020581     99.935616   
7  gi4e_detected_eyes  DenseNet  0.004170     99.965010   
8  gi4e_detected_eyes       VGG  0.025227     95.954054   

                 total_time  
0 0 days 00:16:43.474281073  
1 0 days 00:17:50.950847864  
2 0 days 00:20:41.351259470  
3 0 days 00:09:57.954471111  
4 0 days 00:11:55.672313929  
5 0 days 00:16:04.275976896  
6 0 days 00:09:37.236201763  
7 0 days 00:10:21.910546303  
8 0 days 00:15:47.006313086  


In [10]:
print(result_df)

              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000008     99.999246   
1           gi4e_full  DenseNet  0.000084     99.997677   
2           gi4e_full       VGG  0.049829     97.527197   
3       gi4e_raw_eyes    ResNet  0.011788     94.142118   
4       gi4e_raw_eyes  DenseNet  0.001661     93.929313   
5       gi4e_raw_eyes       VGG  0.075721     74.313858   
6  gi4e_detected_eyes    ResNet  0.020581     99.935616   
7  gi4e_detected_eyes  DenseNet  0.004170     99.965010   
8  gi4e_detected_eyes       VGG  0.025227     95.954054   

                 total_time  
0 0 days 00:16:43.474281073  
1 0 days 00:17:50.950847864  
2 0 days 00:20:41.351259470  
3 0 days 00:09:57.954471111  
4 0 days 00:11:55.672313929  
5 0 days 00:16:04.275976896  
6 0 days 00:09:37.236201763  
7 0 days 00:10:21.910546303  
8 0 days 00:15:47.006313086  


# Embedded Classification

- Load the dataset

In [ ]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Define the model

In [11]:
embedded_models = [md.FeatureExtractor(model) for model in models]
classifier_models = [md.Classifier(model, 103) for model in embedded_models]


- Load the trained weight to the model

In [12]:
# ignore

- Transform the dataset to the embedded data

In [16]:
classifier_df = pd.DataFrame(columns=['key', 'dataset', 'model'])

# Loop through datasets and models to create the classifier_df
print('Starting to get features for datasets...')
for name, dataset in datasets.items():
	for model in embedded_models:
		print(f'Getting features for {name} dataset with {model._get_name()}')

		# get the classifier model from the model
		classifier_model = next((m for m in classifier_models if m.backbone._get_name() == model._get_name()), None)
		if classifier_model is None:
			print(f'Classifier model not found for {model._get_name()}')
			continue

		model_dataset = ds.EmbeddedDataset(dataset, model)

		classifier_df = pd.concat([classifier_df, pd.DataFrame({
				'key': [f'{name}_{model._get_name()}'],
				'model': [classifier_model],
				'dataset': [model_dataset]
		})], ignore_index=True)

		print(f'Finished getting features for {name} dataset with {model._get_name()}')


Starting to get features for datasets...
Getting features for gi4e_full dataset with FeatureExtractor(ResNet)


<All keys matched successfully>


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

- Train all defined model on each registered datasets

In [ ]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for index, row in classifier_df.iterrows():
	dataset = row['dataset']
	model = row['model']
	key = row['key']

	print(f'Running {key} dataset with {model._get_name()}')
	# do the train
	start_time = time.time()
	scored, loss = train_process(dataset, model)
	end_time = time.time()
	total_time = end_time - start_time
	print(f'Finished {key} dataset with {model._get_name()}')
	print('----------------------')

	# save the result
	result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model._get_name()],
			'dataset': [dataset.__class__.__name__],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
	})], ignore_index=True)
print('Finished training all models')


- Print the result

In [ ]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


In [ ]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

for model in models:
  # explore the model
	print(f'Model: {model.__class__.__name__}')
	
	# explain the model
	print(f'Number of layers: {len(list(model.children()))}')
	print(model)